# Simple example of using the `Mf6Cts` package

This notebook will demo a very simple use case of the `Mf6Cts` contaminant treatment system (CTS) package.  A CTS instance looks like this:

<img src="fig1.png" width=700 height=700 />

Essentially, a CTS can have multiple extractions wells and multiple injection wells.  The `Mf6Cts` implementation allows both "WEL" and "MAW" type boundaries for a CTS instance.  

The implementation herein allows multiple CTS instances, and the configuration and treatment efficiency of each CTS instance can change at the stress period level.  This is all similar to the CTS functionality implemented in MT3D-USGS.

What our implementation does that is different is 
 - it automatically balances extraction flows and injection flows and 
 - support unstructured grids.  
 
The former is important when the model may not be able to meet the requested extraction (during scenario testing, management optimization, etc).

Let's see this thing in action!


First import the usuals

In [ ]:
import os
import sys
import shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


And we need to get access to the autotesting script and the `Mf6Cts` class itself:

In [ ]:
sys.path.append(os.path.join("..","autotest"))
sys.path.append(os.path.join("..","mf6cts"))
import flopy #the flopy scr is in the autotest dir...

In [ ]:
import mf6cts # the src file holding the Mf6Cts class

In [ ]:
import cts_mf6_test # the autotesting script

And we need to know the name of the mf6 shared lib and executable files for the current operating system:

In [ ]:
lib_name = cts_mf6_test.lib_name
mf6_bin = cts_mf6_test.mf6_bin

Now we will use a handy-dandy function in the autotest script to setup the flow and transport models for us.  This model will have two CTS instances and each instance will have a time-varying (random) efficiencies centered around the values passed below.  dissolved phase mass will be introduced via the upgradient GHB-type boundary condition

In [ ]:
org_sim_ws = "fivespot"
np.random.seed(111)
cts_mf6_test.setup_five_spotish(plot=False, sim_ws=org_sim_ws, simple_pattern=True, eff1=0.5, eff2=0.7, nlay=1,ghb_source=1)


That function essentially built and ran the flow and transport models for us (for testing purposes) and also wrote the CTS input file

The flow-model subdirectory:

In [ ]:
os.listdir("fivespot")

The transport-model subdirectory:

In [ ]:
os.listdir("fivespot_t")

Ooohh - let's check out "model.cts"!

In [ ]:
lines = open(os.path.join("fivespot_t","model.cts")).readlines()
_ =[print(line.strip()) for line in lines]

We see the each CTS instance has a period block for each stress period that it is active and the each period block can have an optional efficiency supplied.  Each period block line lists the package type (in this case "wel"), the package instance name ("wel_0"), an in-out flag (either "in" or "out") and then index information that describes where in the model the CTS member is located.  For WEL in structured grids, this is the layer-row-column info.  For WEL in unstructed grids, this is the node number.  For MAW-type boundaries, this is the "welno" value. 

Now lets check out the standard MF6 solution results:

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_ws="fivespot")
m = sim.get_model("gwf")
widx = [i[0] for i in m.get_package("wel").stress_period_data.data[1]]
hds = flopy.utils.HeadFile(os.path.join("fivespot","gwf.hds"))
ucn = flopy.utils.HeadFile(os.path.join("fivespot_t","gwt.ucn"),text="concentration")
fig,ax = plt.subplots(1,1,figsize=(6,6))
carr = ucn.get_data()[0]
carr[carr<=0.0001] = np.nan
c = ax.imshow(carr,cmap="jet",alpha=0.5)
plt.colorbar(c,ax=ax)
lb = ax.contour(hds.get_data()[0],levels=5,colors='0.5')
ax.clabel(lb,fontsize=15)
for w in widx:
    ax.scatter(w[2],w[1],marker="^",c="k")
garr = np.zeros((m.dis.nrow.data,m.dis.ncol.data))
for idx in m.get_package("ghb").stress_period_data.data[1]:
    garr[idx[0][1],idx[0][2]] = idx[2]
garr[garr == 0] = np.nan
ax.imshow(garr,cmap="cool")


The triangles are the WEL locations.  Two extraction wells are shown near the center-right of the model domain, the injections wells are along the edges of the domain.  The effect the of the pump-and-treat system can be seen in the head contours and the concentration distribution.  Remember, this is just the standard MF6 flow and transport solution - no CTS-specific flow balancing or treatment efficiency has been applied.

Now let's check out the global water balance info:

In [ ]:
inc,cum = flopy.utils.Mf6ListBudget(os.path.join("fivespot","gwf.lst")).get_dataframes()

In [ ]:
inc.loc[:,inc.columns.str.contains("WEL")]

Ruh roh - we were injecting more than we were extracting, meaning our CTS instance were not "closed-loop". #sad

### CTS time!

First we need to copy the shared MF6 library file to both the flow and transport model directories:

In [ ]:
shutil.copy2(lib_name, os.path.join("fivespot_t", os.path.split(lib_name)[-1]))
shutil.copy2(lib_name, os.path.join("fivespot", os.path.split(lib_name)[-1]))

Now we can instantiate an `Mf6Cts` instance:

In [ ]:
mf = mf6cts.Mf6Cts("model.cts",os.path.split(lib_name)[-1],"fivespot_t","fivespot")

Solve the flow-model, balancing the each CTS instance's extraction vs injection rates:

In [ ]:
mf.solve_gwf()

Let's inspect some of the CTS-specific CSV files that were created:

In [ ]:
pd.read_csv(os.path.join("fivespot","gwf_cts_flow_node_summary.csv"))

In [ ]:
pd.read_csv(os.path.join("fivespot","gwf_cts_flow_system_summary.csv"))

We can see that the CTS instances were (purposefully) unable to meet the requested extraction rates.  This resulted in turning down the injection rates so that we arent injecting more water than we are extracting for each CTS instance, for each stress period/time step

In [ ]:
inc,cum = flopy.utils.Mf6ListBudget(os.path.join("fivespot","gwf.lst")).get_dataframes()
inc.loc[:,inc.columns.str.contains("WEL")]

Now we see that the extraction and injection rates balance according to the MODFLOW global water budget info - ah yeah!

Now to run the transport solution using the flow-balanced CTS results, we need to copy the flow model binary output files to the tranport model directory:

In [ ]:
shutil.copy2(os.path.join("fivespot", "gwf.hds"), os.path.join("fivespot_t" "gwf.hds"))
shutil.copy2(os.path.join("fivespot", "gwf.bud"), os.path.join("fivespot_t", "gwf.bud"))

Now solve the transport model and apply the requested efficiency factor to each CTS instance for each stress period:

In [ ]:
mf.solve_gwt()

Finalize, which releases control over the file handles:

In [ ]:
mf.finalize()

Now lets visualize the results of the flow and transport models that were solved using the CTS-specific results...

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_ws="fivespot")
m = sim.get_model("gwf")
widx = [i[0] for i in m.get_package("wel").stress_period_data.data[1]]
hds = flopy.utils.HeadFile(os.path.join("fivespot","gwf.hds"))
ucn = flopy.utils.HeadFile(os.path.join("fivespot_t","gwt.ucn"),text="concentration")
fig,ax = plt.subplots(1,1,figsize=(6,6))
carr = ucn.get_data(kstpkper=(0,7))[0]
carr[carr<=0.0001] = np.nan
c = ax.imshow(carr,cmap="jet",alpha=0.5)
plt.colorbar(c,ax=ax)
lb = ax.contour(hds.get_data(kstpkper=(0,7))[0],levels=5,colors='0.5')
ax.clabel(lb,fontsize=15)
for w in widx:
    ax.scatter(w[2],w[1],marker="^",c="k")
garr = np.zeros((m.dis.nrow.data,m.dis.ncol.data))
for idx in m.get_package("ghb").stress_period_data.data[1]:
    garr[idx[0][1],idx[0][2]] = idx[2]
garr[garr == 0] = np.nan
ax.imshow(garr,cmap="cool")


We can see the effect of the flow balancing (compared to above) in the head contours.  We can also see the effect of the treatment effiency in the concentration distribution, especially near the injection wells.

Let's finish up by looking at the transport model CTS summary CSV files:

In [ ]:
pd.read_csv(os.path.join("fivespot_t","gwt_cts_node_summary.csv"))

In [ ]:
df = pd.read_csv(os.path.join("fivespot_t","gwt_cts_system_summary.csv"))
df

In [ ]:
fig,axes = plt.subplots(2,1,figsize=(6,6))
for ax,cts in zip(axes,df.cts_system.unique()):
    cdf = df.loc[df.cts_system==cts,:].copy()
    cdf.sort_values(by="stress_period",inplace=True)
    ax.plot(cdf.stress_period,cdf.concen_injected,"g")
    axt = plt.twinx(ax)
    axt.scatter(cdf.stress_period,cdf.requested_efficiency,c='m')
    ax.set_title("cts:{0}".format(cts),loc="left",fontsize=10)
    ax.set_xlabel('stress period',fontsize=10)
    ax.set_ylabel('concentration',fontsize=10,color="g")
    ax.set_ylim(0,1)
    axt.set_ylim(0,1)
    axt.set_ylabel("efficiency",fontsize=10,color="m")
    ax.grid()